In [2]:
import blankly
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import LSTM
import torch.optim as optim
from torch.autograd import Variable



In [3]:
def episode_gen(data, seq_length,output_size):
    x = []
    y = []
    #Loop through data, adding input data to x array and output data to y array
    for i in range(len(data)-seq_length):
        _x = data[i:(i+seq_length - output_size)]
        _y = data[i+seq_length - output_size:i + seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

In [4]:
def init_NN(symbol, state: blankly.StrategyState):
    interface = state.interface
    resolution = state.resolution
    variables = state.variables

    #Get price data
    variables['history'] = interface.history(symbol, 300, resolution, return_as='list')['close']
    rsi = blankly.indicators.rsi(state.variables['history'])
    macd = blankly.indicators.macd(state.variables['history'])

    seq_length = 8
    output_size = 3

    x = [variables['history'][i] / variables['history'][i-1] for i in range(25,len(variables['history']))]
    x, y = episode_gen(x, seq_length, output_size)
    y = Variable(torch.Tensor(np.array(y))).unsqueeze(0)
    #RSI data gathering
    x_rsi = rsi[11:]
    x_rsi,_ = episode_gen(x_rsi,seq_length, output_size)

    #MACD data gathering
    macd_vals,_ = episode_gen(macd[0], seq_length, output_size)
    macd_signals,_ = episode_gen(macd[1],seq_length, output_size)

    x_agg = np.zeros((len(macd_signals),seq_length-output_size, 4))
    for i in range(len(macd_signals)):
      for j in range(seq_length - output_size):
        x_agg[i][j][0] = x[i][j]
        x_agg[i][j][1] = x_rsi[i][j]
        x_agg[i][j][2] = macd_vals[i][j]
        x_agg[i][j][3] = macd_signals[i][j]
    x_tot = Variable(torch.Tensor(x_agg))
    
    #begin training
    num_epochs = 10000
    learning_rate = 0.0003


    state.lstm = LSTM(4,20, batch_first = True)
    state.lin = nn.Linear(20,3)
    criterion = torch.nn.MSELoss()    
    optimizer = torch.optim.Adam([
                {'params': state.lstm.parameters()},
                {'params': state.lin.parameters()}
            ], lr=learning_rate)

    # Train the model
    for epoch in range(num_epochs):
        #run model
        outputs, (h_n, c_n) = state.lstm(x_tot)
        out = state.lin(h_n)
        out = 0.5 + F.sigmoid(out)
        optimizer.zero_grad()
        
        loss = criterion(out, y) 
        
        loss.backward() 
        
        optimizer.step() 

        if epoch % 500 == 0:
          print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))
    
    state.lastthree = [[0,0],[0,0],[0,0]] 

In [5]:
def price_lstm(price,symbol,state: blankly.StrategyState):
    state.variables['history'].append(price) #Add latest price to current list of data

    into = [state.variables['history'][i]/state.variables['history'][i-1] for i in range(-5,0)]
    rsi = blankly.indicators.rsi(state.variables['history'])
    rsi_in = np.array(rsi[-5:])
    macd = blankly.indicators.macd(state.variables['history'])
    macd_vals = np.array(macd[0][-5:])
    macd_signals = np.array(macd[1][-5:])

    
    pred_in = np.zeros((1,len(into),4))
    for i in range(len(into)):
      pred_in[0][i][0] = into[i]
      pred_in[0][i][1] = rsi_in[i]
      pred_in[0][i][2] = macd_vals[i]
      pred_in[0][i][3] = macd_signals[i]
    pred_in = torch.Tensor(pred_in)
  
    print(price)

   
    out,(h,c) = state.lstm(pred_in)
    out = state.lin(h)
    out = 0.5 + F.sigmoid(out)
    
   
    state.lastthree[0][0]+=out[0][0][0]
    state.lastthree[0][1]+=1
    state.lastthree[1][0]+=out[0][0][1]
    state.lastthree[1][1]+=1
    state.lastthree[2][0]+=out[0][0][2]
    state.lastthree[2][1]+=1

    
    priceavg = state.lastthree[0][0]/state.lastthree[0][1]

    curr_value = blankly.trunc(state.interface.account[state.base_asset].available, 2)
    if priceavg > 1:
        buy = blankly.trunc(state.interface.cash  * 2 * (priceavg.item() - 1)/price, 2) 
        if buy > 0:
          state.interface.market_order(symbol, side='buy', size=buy)
    elif curr_value > 0:
         cv =  blankly.trunc(curr_value  * 2 * (1 - priceavg.item()),2)
         if cv > 0:
          state.interface.market_order(symbol, side='sell', size=cv)

    print("prediction for price --",priceavg)
    state.lastthree = [state.lastthree[1], state.lastthree[2], [0,0]]

In [15]:
exchange = blankly.Alpaca() #Connect to API
strategy = blankly.Strategy(exchange) #Initialize a Blankly strategy
strategy.add_price_event(price_lstm, symbol='AAPL', resolution='1d', init=init_NN)
results = strategy.backtest(to='1y', initial_values={'USD': 10000}) #Backtest one year starting with $10,000
print(results)

INFO: No portfolio name to load specified, defaulting to the first in the file: (another cool portfolio). This is fine if there is only one portfolio in use.


No cached data found for AAPL from: 1683944667.2364469 to 1715394267.236473 at a resolution of 86400 seconds.

Backtesting...
Epoch: 0, loss: 0.00243
Epoch: 500, loss: 0.00041
Epoch: 1000, loss: 0.00038
Epoch: 1500, loss: 0.00036
Epoch: 2000, loss: 0.00033
Epoch: 2500, loss: 0.00030
Epoch: 3000, loss: 0.00026
Epoch: 3500, loss: 0.00022
Epoch: 4000, loss: 0.00019
Epoch: 4500, loss: 0.00017
Epoch: 5000, loss: 0.00016
Epoch: 5500, loss: 0.00015
Epoch: 6000, loss: 0.00015
Epoch: 6500, loss: 0.00014
Epoch: 7000, loss: 0.00014
Epoch: 7500, loss: 0.00013
Epoch: 8000, loss: 0.00013
Epoch: 8500, loss: 0.00013
Epoch: 9000, loss: 0.00012
Epoch: 9500, loss: 0.00012
172.07
prediction for price -- tensor(1.0027, grad_fn=<DivBackward0>)
172.07
prediction for price -- tensor(1.0072, grad_fn=<DivBackward0>)
172.07
prediction for price -- tensor(1.0068, grad_fn=<DivBackward0>)
172.07
prediction for price -- tensor(1.0098, grad_fn=<DivBackward0>)
172.69
prediction for price -- tensor(1.0101, grad_fn=<Div